# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [1]:
import autowig

We need then to install and compile the *C++* library.
This is done using the `cpp` target in **SCons**.

In [2]:
%%bash
scons cpp -C basic

scons: Entering directory `/home/pfernique/Desktop/AutoWIG/doc/examples/basic'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o build/cpp/binomial.os -c -x c++ -std=c++0x -Wwrite-strings -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -I/home/pfernique/.miniconda2/envs/autowig/include -I/home/pfernique/.miniconda2/envs/autowig/include/python2.7 build/cpp/binomial.cpp
scons: `cpp' is up to date.
scons: done building targets.


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [3]:
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [4]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [5]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg, module = 'basic/src/py/_module.cpp',
                                  decorator = 'basic/src/py/basic/_module.py',
                                  prefix = 'wrapper_')

The wrappers are only generated in-memory.
We therefore need to write them on the disk to complete the process.

In [6]:
wrappers.write()

Once the wrappers are written on disk, we need to compile the *Python* bindings using the `py` target in **SCons**.

In [7]:
%%bash
scons py -C basic

scons: Entering directory `/home/pfernique/Desktop/AutoWIG/doc/examples/basic'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: `py' is up to date.
scons: done building targets.


The *Python* package is now ready to be installed.

In [8]:
%%bash
pip install -e basic

Obtaining file:///home/pfernique/Desktop/AutoWIG/doc/examples/basic
  Running setup.py develop for basic


And we can hereafter use the *C++* library in the *Python* interpreter

In [ ]:
import basic
binomial = basic.BinomialDistribution(1, .5)

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0

In [ ]:
binomial.pmf(0)

In [ ]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error.message
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')